In [1]:
import numpy as np
import pandas as pd
import loss_constraint as lc
import algo_update as al 
import matplotlib.pyplot as plt 
import os
from os import walk
import data_process_update as dp

####  测试   
##### 正常运行会在result\case3\text生成前10个数据的结果

In [2]:
directory = "test"
parent_dir = r'E:\python project\生产调度\result\case3'
test_path = os.path.join(parent_dir, directory)
os.mkdir(test_path) 

length_set, file_name_set, final_path_set = dp.path_process(r'E:\python project\生产调度\100组数据集\数据集')

param=[]
for i in range(0,100):
    args = [final_path_set[i],test_path,length_set[i],file_name_set[i],100,100,0.1,0.6,False,None,'cross_op_1','auto_mutation',0.7]
    param.append(args)


In [3]:
from tqdm import tqdm
import multiprocessing as mp
num_cores = int(mp.cpu_count())
print("本地计算机有: " + str(num_cores) + " 核心")

pbar = tqdm(total=10)
pbar.set_description('任务处理进程')
update = lambda *args: pbar.update()
pool = mp.Pool(num_cores)
for args in param[0:10]:
	pool.apply_async(dp.process, args, callback=update)
pool.close()
pool.join()


本地计算机有: 4 核心


任务处理进程: 100%|██████████| 10/10 [01:07<00:00,  5.75s/it]

#### 全量运行

In [5]:
length_set, file_name_set, final_path_set = dp.path_process(r'E:\python project\生产调度\100组数据集\数据集')
res_path = r'E:\python project\生产调度\result\case3'
N_set = [min(10*int(length_set[i]/10) + 200,1000) for i in range(0,100)]
T_set = [min(20*int(length_set[i]/10) + 400, 5000) for i in range(0,100)]

param=[]
for i in range(0,100):
    args = [final_path_set[i],res_path,length_set[i],file_name_set[i],N_set[i],T_set[i],0.1,0.6,False,None,'cross_op_1','auto_mutation',0.7]
    param.append(args)

In [ ]:
from tqdm import tqdm
import multiprocessing as mp
num_cores = int(mp.cpu_count())
print("本地计算机有: " + str(num_cores) + " 核心")

pbar = tqdm(total=100)
pbar.set_description('任务处理进程')
update = lambda *args: pbar.update()
pool = mp.Pool(num_cores)
for args in param:
	pool.apply_async(dp.process, args, callback=update)
pool.close()
pool.join()

#### 结果写入

In [11]:
path = r'E:\python project\生产调度\result\case3'
os.chdir(path)

# 遍历文件
res = []
for (dir_path, dir_name, file_name) in walk(path,topdown=True):
    res.extend(file_name)

# 获取excel文件地址
last_name = [i.split('.')[1] for i in res]
xlsx_res = []
for i in range(0,len(last_name)):
    if last_name[i] == 'xlsx':
        xlsx_res.append(res[i])

length = np.array([eval(i.split('_')[1].split('.')[0]) for i in xlsx_res])
id_sort = np.argsort(length)
file_name_list = []
for i in id_sort:
    file_name_list.append(xlsx_res[i])

length = length[id_sort]

# 拼接全路径
final_path = []
for file_name in file_name_list:
    tmp_final_path = os.path.join(path, file_name)
    final_path.append(tmp_final_path)

In [13]:
writer = pd.ExcelWriter(r"E:\python project\生产调度\result\case3\case_3_result.xlsx")

for i in range(0,len(final_path)):
    path = final_path[i]
    df = pd.read_excel(path)

    col = []
    for j in range(1,df.shape[1]+1):
        col.append('Variable ' + str(j))
    df.columns = col
    df.to_excel(writer,file_name_list[i].split('.')[0],index=False)

writer.save()